In [1]:
import pandas as pd
import plotly.express as px

columns = [
        # 'join_id',
        'dt','chain', 'active_secs_per_day'
        ,'num_l1_txs_inbox','num_l1_txs_output','calldata_bytes_l1_inbox'
        ,'avg_l1_gas_price_on_l1_inbox','avg_l1_gas_price_on_l1_output','avg_l1_gas_price_on_l2'
        ,'l1_gas_used_inbox','l1_gas_used_output','l1_gas_used_combined'
        ,'l1_eth_fees_inbox','l1_eth_fees_output','l1_eth_fees_combined'
        ,'l1_contrib_l2_eth_fees_per_day','l2_contrib_l2_eth_fees_per_day'
        # ,'l2_num_raw_txs_per_day'
        ,'l2_num_txs_per_day', 'l2_num_attr_deposit_txs_per_day', 'l2_num_user_deposit_txs_per_day'
        ,'l1_gas_used_on_l2','calldata_bytes_l2_per_day','calldata_gas_l2_per_day'
        ,'l1_gas_used_user_txs_l2_per_day','calldata_bytes_user_txs_l2_per_day'
        ,'l2_gas_used','l2_eth_fees_per_day'
        ]

In [2]:
#Unify datasets
dunedf = pd.read_csv('csv_inputs/op_chain_gas_economics_dune_query_2453515_dt2023_09_19.csv') #https://dune.com/queries/2453515 --last 90 days
# goldskydf = pd.read_csv('csv_inputs/op_chain_gas_economics_goldsky_zora_pgn_dt2023_09_12.csv') #requires auth - https://dash.goldsky.com/question/9-op-chains-activity-by-period

In [3]:
# Map Chain Names
chain_mappings = {
    'zora': 'Zora Network',
    'pgn': 'Public Goods Network',
    # Add more mappings as needed
}
# goldskydf['chain'] = goldskydf['chain'].replace(chain_mappings)
dunedf['name'] = dunedf['name'].replace(chain_mappings)
# print(dunedf.columns)

In [4]:
#Configure
#rename cols
dunedf = dunedf.rename(columns={
                        'name':'chain'
                        })
difference = set(columns) - set(dunedf.columns)
result = list(difference)

print(result)


[]


In [5]:
#Generate Join ID Column
# dunedf['join_id'] = dunedf['dt'].astype(str).str[:10].str.cat(dunedf['chain'].astype(str).str.lower())
# goldskydf['join_id'] = goldskydf['dt'].astype(str).str[:10].str.cat(goldskydf['chain'].astype(str).str.lower())

# display(dunedf.head(10))
# print(dunedf.columns)
# print('---')
# display(goldskydf.head(10))
# print(goldskydf.columns)

combo_df = dunedf.copy()#merge(goldskydf, on='join_id', how='left')

# display(combo_df)

# for c in columns:
#         try:
#                 combo_df[c] = combo_df[c+'_x'].combine_first(combo_df[c+'_y']) #pick first non-null
#         except:
#                 combo_df[c] = combo_df[c] #nada

combo_df['dt'] = pd.to_datetime(combo_df['dt'])
combo_df = combo_df[columns]

combo_df['gas_compression_ratio'] = combo_df['l1_gas_used_inbox'] / combo_df['l1_gas_used_on_l2']
combo_df['bytes_compression_ratio'] = combo_df['calldata_bytes_l1_inbox'] / combo_df['calldata_bytes_l2_per_day']
combo_df['dt_rank'] = combo_df['dt'].rank(method='dense', ascending=False).astype('int')

In [6]:
display( combo_df.sample(5) )

combo_df.to_csv('outputs/op_chain_gas_economics_sample.csv')

,dt,chain,active_secs_per_day,num_l1_txs_inbox,num_l1_txs_output,calldata_bytes_l1_inbox,avg_l1_gas_price_on_l1_inbox,avg_l1_gas_price_on_l1_output,avg_l1_gas_price_on_l2,l1_gas_used_inbox,...,l1_gas_used_on_l2,calldata_bytes_l2_per_day,calldata_gas_l2_per_day,l1_gas_used_user_txs_l2_per_day,calldata_bytes_user_txs_l2_per_day,l2_gas_used,l2_eth_fees_per_day,gas_compression_ratio,bytes_compression_ratio,dt_rank
161,2023-08-09 00:00:00+00:00,Base,86400.0,1431,24,74396284.0,23.617619,20.593268,23.712454,1.216806e+09,...,2.051333e+09,156005725.0,1.259559e+09,2.051333e+09,143579019.0,9.987607e+10,115.077455,0.593178,0.476882,41
175,2023-08-06 00:00:00+00:00,Public Goods Network,86400.0,145,24,1887171.0,14.472291,14.774123,12.696032,3.318583e+07,...,8.042440e+05,11322505.0,8.397921e+07,8.042440e+05,49345.0,2.202757e+09,0.036471,41.263388,0.166674,44
40,2023-09-08 00:00:00+00:00,Base,86400.0,1428,24,64497364.0,15.453782,15.158924,15.295237,1.058815e+09,...,1.611126e+09,119829129.0,9.883465e+08,1.611126e+09,108218332.0,2.211997e+11,33.423184,0.657189,0.538244,11
329,2023-06-24 00:00:00+00:00,Base,NaN,1424,24,2038403.0,14.842148,14.638570,NaN,6.230355e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87
255,2023-07-17 00:00:00+00:00,Public Goods Network,86400.0,145,24,1861642.0,21.050896,19.874771,25.238927,3.277944e+07,...,1.717600e+05,11243646.0,8.329290e+07,1.717600e+05,10386.0,2.177028e+09,0.006875,190.844457,0.165573,64


In [7]:
bl_gas_ratio = combo_df['l1_gas_used_inbox'].sum() / combo_df['l1_gas_used_on_l2'].sum()
bl_bytes_ratio = combo_df['calldata_bytes_l1_inbox'].sum() / combo_df['calldata_bytes_l2_per_day'].sum()
print('Blended gas compression ratio: ' + str(round(bl_gas_ratio*100,1)) + '%')
print('Blended bytes compression ratio: ' + str(round(bl_bytes_ratio*100,1)) + '%')

Blended gas compression ratio: 62.4%
Blended bytes compression ratio: 46.1%


In [8]:
# Create the scatter plot
combo_df_plot = combo_df[
                        (~combo_df['gas_compression_ratio'].isna()) 
                         & (combo_df['gas_compression_ratio']<1) 
                         & (combo_df['active_secs_per_day'] == 86400)
                         ]

# combo_df_plot = combo_df[(~combo_df['bytes_compression_ratio'].isna())
#                          & (combo_df['active_secs_per_day'] == 86400)]

display(combo_df_plot)

fig = px.scatter(combo_df_plot,
                 x='calldata_gas_l2_per_day',#'l1_gas_used_on_l2',
                 y='l1_gas_used_inbox',
                 size = 'gas_compression_ratio',
                 color='chain',
                 title='L1 Gas Used on each Layer by chain')

# Display the plot
fig.show()

##############

fig_bytes = px.scatter(combo_df_plot,
                 x='calldata_gas_l2_per_day',#'calldata_bytes_l2_per_day',
                 y='calldata_bytes_l1_inbox',
                 size = 'bytes_compression_ratio',
                 color='chain',
                 title='Calldata Bytes on each Layer by chain')

fig_bytes.show()

,dt,chain,active_secs_per_day,num_l1_txs_inbox,num_l1_txs_output,calldata_bytes_l1_inbox,avg_l1_gas_price_on_l1_inbox,avg_l1_gas_price_on_l1_output,avg_l1_gas_price_on_l2,l1_gas_used_inbox,...,l1_gas_used_on_l2,calldata_bytes_l2_per_day,calldata_gas_l2_per_day,l1_gas_used_user_txs_l2_per_day,calldata_bytes_user_txs_l2_per_day,l2_gas_used,l2_eth_fees_per_day,gas_compression_ratio,bytes_compression_ratio,dt_rank
0,2023-09-18 00:00:00+00:00,Base,86400.0,1765,24,126965466.0,19.491563,14.246336,20.030596,2.061455e+09,...,4.034988e+09,194285446.0,1.477366e+09,4.034988e+09,182163366.0,2.206090e+11,93.363777,0.510895,0.653500,1
1,2023-09-18 00:00:00+00:00,OP Mainnet,86400.0,546,24,62864270.0,15.981227,15.227230,15.548883,1.014413e+09,...,1.515646e+09,131183212.0,1.056304e+09,1.515646e+09,119737172.0,2.211299e+11,56.276676,0.669294,0.479210,1
2,2023-09-18 00:00:00+00:00,Zora Network,86400.0,147,24,9501709.0,14.265629,16.688630,13.736038,1.546667e+08,...,1.945070e+08,22572491.0,1.733268e+08,1.945070e+08,11337899.0,1.062391e+10,3.380593,0.795173,0.420942,1
4,2023-09-17 00:00:00+00:00,Base,86400.0,1534,24,114930716.0,9.161427,8.775919,9.087644,1.864776e+09,...,3.609370e+09,194144328.0,1.440871e+09,3.609370e+09,182047165.0,2.202859e+11,39.379208,0.516649,0.591986,2
5,2023-09-17 00:00:00+00:00,OP Mainnet,86400.0,467,24,53929488.0,9.179985,8.717410,9.114567,8.702823e+08,...,1.332877e+09,117005933.0,9.119203e+08,1.332877e+09,105560593.0,2.215978e+11,53.443908,0.652935,0.460912,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2023-06-25 00:00:00+00:00,OP Mainnet,86400.0,878,24,96019912.0,13.110790,12.860897,12.998240,1.550488e+09,...,2.544884e+09,217137993.0,1.667716e+09,2.544884e+09,205688377.0,1.623467e+11,27.251725,0.609257,0.442207,86
328,2023-06-24 00:00:00+00:00,OP Mainnet,86400.0,781,24,85566933.0,15.243767,17.169366,14.978029,1.381636e+09,...,2.227030e+09,186589428.0,1.465352e+09,2.227030e+09,175148720.0,1.313212e+11,26.593378,0.620394,0.458584,87
331,2023-06-23 00:00:00+00:00,OP Mainnet,86400.0,914,24,101164449.0,19.046680,17.070065,18.736747,1.633251e+09,...,2.649824e+09,222876030.0,1.731425e+09,2.649824e+09,211437942.0,1.583357e+11,40.881082,0.616362,0.453905,88
334,2023-06-22 00:00:00+00:00,OP Mainnet,86400.0,912,24,100270453.0,16.844497,15.514757,16.707737,1.618922e+09,...,2.651979e+09,223576530.0,1.739353e+09,2.651979e+09,212120234.0,1.919369e+11,35.849179,0.610458,0.448484,89


In [9]:
new_data = combo_df[combo_df['active_secs_per_day'] == 86400].copy()
display(new_data.sample(5))

,dt,chain,active_secs_per_day,num_l1_txs_inbox,num_l1_txs_output,calldata_bytes_l1_inbox,avg_l1_gas_price_on_l1_inbox,avg_l1_gas_price_on_l1_output,avg_l1_gas_price_on_l2,l1_gas_used_inbox,...,l1_gas_used_on_l2,calldata_bytes_l2_per_day,calldata_gas_l2_per_day,l1_gas_used_user_txs_l2_per_day,calldata_bytes_user_txs_l2_per_day,l2_gas_used,l2_eth_fees_per_day,gas_compression_ratio,bytes_compression_ratio,dt_rank
257,2023-07-16 00:00:00+00:00,Zora Network,86400.0,163,24,5561592.0,16.914880,16.197011,17.339785,9.210061e+07,...,7.471650e+07,16675783.0,1.399609e+08,7.471650e+07,5443783.0,5.386712e+09,2.128575,1.232668,0.333513,65
8,2023-09-16 00:00:00+00:00,Base,86400.0,1821,24,153218173.0,10.723150,10.975514,10.646060,2.480891e+09,...,4.900869e+09,203808757.0,1.568070e+09,4.900869e+09,191696202.0,2.202168e+11,63.970008,0.506215,0.751774,3
34,2023-09-10 00:00:00+00:00,Zora Network,86400.0,184,24,14309930.0,9.740409,9.415879,9.618155,2.321615e+08,...,3.441251e+08,32649091.0,2.427135e+08,3.441251e+08,21407411.0,1.705784e+10,5.417414,0.674643,0.438295,9
35,2023-09-10 00:00:00+00:00,Public Goods Network,86400.0,145,24,1865405.0,9.737466,9.618426,9.473016,3.284035e+07,...,2.142080e+05,11257144.0,8.328854e+07,2.142080e+05,15064.0,2.186807e+09,0.015078,153.310558,0.165709,9
164,2023-08-08 00:00:00+00:00,OP Mainnet,86400.0,1594,24,187088490.0,23.122305,24.298956,23.104643,3.017709e+09,...,4.659603e+09,376881201.0,3.148146e+09,4.659603e+09,365475993.0,1.995162e+11,94.102721,0.647632,0.496412,42


In [10]:
selected_columns = [
    'l2_num_txs_per_day' #, 'l2_num_attr_deposit_txs_per_day', 'l2_num_user_deposit_txs_per_day'
    # , 'l1_gas_used_on_l2'
    , 'calldata_gas_l2_per_day'
    , 'calldata_bytes_l2_per_day'
    # , 'l1_gas_used_user_txs_l2_per_day', 'calldata_bytes_user_txs_l2_per_day'
    , 'l1_gas_used_inbox'
]

missing_values_in_selected_columns = new_data[selected_columns].isnull().sum()
missing_values_in_selected_columns

l2_num_txs_per_day           0
calldata_gas_l2_per_day      0
calldata_bytes_l2_per_day    0
l1_gas_used_inbox            0
dtype: int64

In [11]:
# Filling the missing values in 'l1_gas_used_on_l2' with 0
# new_data['l1_gas_used_on_l2'].fillna(0, inplace=True)
new_data['calldata_gas_l2_per_day'].fillna(0, inplace=True)
new_data['calldata_bytes_l2_per_day'].fillna(0, inplace=True)
# new_data['l1_gas_used_user_txs_l2_per_day'].fillna(0, inplace=True)
# new_data['calldata_bytes_user_txs_l2_per_day'].fillna(0, inplace=True)

# Verifying that there are no more missing values in the selected columns
missing_values_in_selected_columns = new_data[selected_columns].isnull().sum()
missing_values_in_selected_columns

l2_num_txs_per_day           0
calldata_gas_l2_per_day      0
calldata_bytes_l2_per_day    0
l1_gas_used_inbox            0
dtype: int64

In [12]:
# Step 2: Feature Selection - Examining the correlation between the potential features and the target variable
correlation_matrix = new_data[selected_columns].corr()

# Getting the correlation of all features with the target variable 'l1_gas_used_inbox'
target_correlation = correlation_matrix['l1_gas_used_inbox']

# Displaying the correlation values
target_correlation

l2_num_txs_per_day           0.913363
calldata_gas_l2_per_day      0.975071
calldata_bytes_l2_per_day    0.970324
l1_gas_used_inbox            1.000000
Name: l1_gas_used_inbox, dtype: float64

In [13]:
# Build new values
new_data['l1_gas_used_per_user_tx'] = new_data['l1_gas_used_user_txs_l2_per_day'] / new_data['l2_num_txs_per_day']
new_data['calldata_bytes_per_user_tx'] = new_data['calldata_bytes_user_txs_l2_per_day'] / new_data['l2_num_txs_per_day']

# display(new_data[new_data['l1_gas_used_user_txs_l2_per_day'] == 0].head(5))

new_data['l1_gas_used_per_user_tx'].fillna(0, inplace=True)
new_data['calldata_bytes_per_user_tx'].fillna(0, inplace=True)

# new_data['l2_num_txs_per_day_sq'] = new_data['l2_num_txs_per_day'] ** 2

# display(new_data.head(5))

In [ ]:
# GPT Built model
from sklearn.model_selection import train_test_split

# Step 3: Data Splitting - Splitting the data into training and testing sets

# Selected features
features = [
    # 'l1_gas_used_user_txs_l2_per_day', 
    # 'calldata_bytes_user_txs_l2_per_day', 
    'l1_gas_used_per_user_tx',
    'calldata_bytes_per_user_tx',
    'l2_num_txs_per_day', 
    # 'l2_num_txs_per_day_sq', 
    'l2_num_user_deposit_txs_per_day', 
    # 'l2_num_attr_deposit_txs_per_day'
]

# Target variable
target = 'l1_gas_used_inbox'

# Splitting the data into training and testing sets (80% training, 20% testing)
X = new_data[features]
y = new_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 4: Model Training - Training a linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

# Step 5: Model Evaluation - Evaluating the model on the testing set
y_pred_linear = linear_regressor.predict(X_test)

# Calculate the model performance
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

mse_linear, r2_linear

In [ ]:
# Extracting the coefficients and intercept from the trained model
intercept = linear_regressor.intercept_
coefficients = linear_regressor.coef_

# Creating a dictionary to hold the feature names and their respective coefficients
feature_coeff_dict = dict(zip(features, coefficients))

# Displaying the intercept and coefficients
intercept, feature_coeff_dict

In [ ]:
from joblib import dump

# Save the model to a file
model_filepath = 'outputs/linear_regression_l1_gas_used_inbox_model.joblib'
dump(linear_regressor, model_filepath)


In [ ]:
# Create a string to represent the regression equation
equation_str = f"l1_gas_used_inbox = {intercept:.2f}"
for feature, coef in feature_coeff_dict.items():
    equation_str += f" + ({coef:.2f}) * {feature}"

# Print out the regression equation
print(equation_str)

In [ ]:
# Find inputs
# Convert the 'dt' column to datetime data type
new_data['dt'] = pd.to_datetime(new_data['dt'])

# Identify the most recent date in the dataset
most_recent_date = new_data['dt'].max()

# Identify the date that is 30 days before the most recent date
date_30_days_ago = most_recent_date - pd.Timedelta(days=30)

# Filter the data to include only the most recent 30 days
recent_30_days_data = new_data[new_data['dt'] > date_30_days_ago]

# Group the data by 'chain' and calculate the average values for each of the input features over the most recent 30 days
average_values_per_chain = recent_30_days_data.groupby('chain')[features].mean()

# Resetting the index to have 'chain' as a column
average_values_per_chain = average_values_per_chain.reset_index()

# Calculate the per transaction values for 'l1_gas_used_on_l2' and 'calldata_bytes_l2_per_day'
# average_values_per_chain['l1_gas_used_user_txs_on_l2_per_tx'] = average_values_per_chain['l1_gas_used_user_txs_l2_per_day'] / average_values_per_chain['l2_num_txs_per_day']
# average_values_per_chain['calldata_bytes_user_txs_l2_per_tx'] = average_values_per_chain['calldata_bytes_user_txs_l2_per_day'] / average_values_per_chain['l2_num_txs_per_day']

# Create a new DataFrame to hold the 'per_tx' values
# average_values_per_tx_chain = average_values_per_chain[['chain', 'l1_gas_used_per_user_tx', 'calldata_bytes_per_user_tx', 'l2_num_txs_per_day']]



# Display the average values per chain
# display(average_values_per_tx_chain)

In [ ]:
import matplotlib.pyplot as plt
# Setting up the plot
plt.figure(figsize=(12, 8))

# List of unique chains
chains = new_data['chain'].unique()

# Looping over each chain to plot the actual vs. predicted values and residuals
for i, chain in enumerate(chains):
    
    # Filtering the data for the current chain
    chain_data = recent_30_days_data[recent_30_days_data['chain'] == chain]
    
    # Making predictions using the new model
    chain_data['predicted_l1_gas_used_inbox_new'] = linear_regressor.predict(chain_data[features])
    
    # Plotting the actual and predicted values
    plt.subplot(2, 2, i+1)
    plt.plot(chain_data['dt'], chain_data['l1_gas_used_inbox'], label='Actual', marker='o', linestyle='-')
    plt.plot(chain_data['dt'], chain_data['predicted_l1_gas_used_inbox_new'], label='Predicted', marker='o', linestyle='-')
    
    # Plotting the residuals
    for j in range(len(chain_data)):
        plt.plot([chain_data['dt'].iloc[j], chain_data['dt'].iloc[j]], 
                 [chain_data['l1_gas_used_inbox'].iloc[j], chain_data['predicted_l1_gas_used_inbox_new'].iloc[j]], 
                 color='gray', linestyle='--', linewidth=0.5)
    
    # Adding labels and title
    plt.xlabel('Date')
    plt.ylabel('L1 Gas Used Inbox')
    plt.title(f'Actual vs Predicted (Chain: {chain})')
    plt.legend()
    plt.grid(True)

# Adjusting the layout to prevent overlapping
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
#Stratify sampling

from sklearn.model_selection import StratifiedShuffleSplit

# Resetting the index of new_data before performing stratified sampling
new_data.reset_index(drop=True, inplace=True)

# Defining the stratified shuffle split object to split the data into training and testing sets with stratification on the 'chain' column
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)


# Splitting the data into training and testing sets using stratified sampling
for train_index, test_index in stratified_split.split(new_data, new_data['chain']):
    stratified_train_data = new_data.loc[train_index]
    stratified_test_data = new_data.loc[test_index]

# Training the linear regression model using the stratified training set
X_train_stratified = stratified_train_data[features]
y_train_stratified = stratified_train_data['l1_gas_used_inbox']
X_test_stratified = stratified_test_data[features]
y_test_stratified = stratified_test_data['l1_gas_used_inbox']

linear_regressor_stratified = LinearRegression()
linear_regressor_stratified.fit(X_train_stratified, y_train_stratified)

# Predicting the target variable on the stratified testing set
stratified_test_data['predicted_l1_gas_used_inbox'] = linear_regressor_stratified.predict(X_test_stratified)

# Setting up the plot for visualization of residuals for each chain
plt.figure(figsize=(12, 8))

# Looping over each chain to plot the actual vs. predicted values and residuals
for i, chain in enumerate(chains):
    
    # Filtering the data for the current chain and sorting it by date for a connected line plot
    chain_data = stratified_test_data[stratified_test_data['chain'] == chain].sort_values(by='dt')
    
    # Plotting the actual and predicted values
    plt.subplot(2, 2, i+1)
    plt.plot(chain_data['dt'], chain_data['l1_gas_used_inbox'], label='Actual', marker='o', linestyle='-')
    plt.plot(chain_data['dt'], chain_data['predicted_l1_gas_used_inbox'], label='Predicted', marker='o', linestyle='-')
    
    # Plotting the residuals
    for j in range(len(chain_data)):
        plt.plot([chain_data['dt'].iloc[j], chain_data['dt'].iloc[j]], 
                 [chain_data['l1_gas_used_inbox'].iloc[j], chain_data['predicted_l1_gas_used_inbox'].iloc[j]], 
                 color='gray', linestyle='--', linewidth=0.5)
    
    # Adding labels and title
    plt.xlabel('Date')
    plt.ylabel('L1 Gas Used Inbox')
    plt.title(f'Actual vs Predicted (Chain: {chain}) - Stratified')
    plt.legend()
    plt.grid(True)

# Adjusting the layout to prevent overlapping
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
# Extracting the coefficients and intercept from the trained model
intercept = linear_regressor_stratified.intercept_
coefficients = linear_regressor_stratified.coef_

# Creating a dictionary to hold the feature names and their respective coefficients
feature_coeff_dict = dict(zip(features, coefficients))

# Displaying the intercept and coefficients
intercept, feature_coeff_dict

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

# Step 1: Create second-degree polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(new_data[features])

# Step 2: Train a linear regression model using the polynomial features
poly_lr = LinearRegression()
poly_lr.fit(X_poly, new_data['l1_gas_used_inbox'])

# Step 3: Make predictions using the polynomial features
new_data['predicted_l1_gas_used_inbox_poly'] = poly_lr.predict(X_poly)

# Step 4: Evaluate the model using suitable metrics
mse_poly = mean_squared_error(new_data['l1_gas_used_inbox'], new_data['predicted_l1_gas_used_inbox_poly'])
mae_poly = mean_absolute_error(new_data['l1_gas_used_inbox'], new_data['predicted_l1_gas_used_inbox_poly'])
r2_poly = r2_score(new_data['l1_gas_used_inbox'], new_data['predicted_l1_gas_used_inbox_poly'])

mse_poly, mae_poly, r2_poly


In [ ]:
# Visualizing the actual vs predicted values for the model with polynomial features
plt.figure(figsize=(10, 6))

# Plotting the actual values
plt.scatter(new_data['dt'], new_data['l1_gas_used_inbox'], label='Actual', alpha=0.5, color='blue')

# Plotting the predicted values from the polynomial regression model
plt.scatter(new_data['dt'], new_data['predicted_l1_gas_used_inbox_poly'], label='Predicted (Poly Reg)', alpha=0.5, color='red')

# Labeling the axes and setting the title
plt.xlabel('Date')
plt.ylabel('L1 Gas Used Inbox')
plt.title('Actual vs Predicted L1 Gas Used Inbox (Polynomial Regression)')

# Displaying the legend
plt.legend()

# Displaying the plot
plt.show()


In [ ]:
# Setting up the plot
plt.figure(figsize=(14, 8))

# List of unique chains
chains = new_data['chain'].unique()

# Plotting the actual and predicted values for each chain in separate subplots
for i, chain in enumerate(chains, 1):
    plt.subplot(2, 2, i)
    
    # Filtering the data for the current chain
    chain_data = new_data[new_data['chain'] == chain]
    
    # Plotting the actual values
    plt.scatter(chain_data['dt'], chain_data['l1_gas_used_inbox'], label='Actual', alpha=0.5, color='blue')
    
    # Plotting the predicted values from the polynomial regression model
    plt.scatter(chain_data['dt'], chain_data['predicted_l1_gas_used_inbox_poly'], label='Predicted (Poly Reg)', alpha=0.5, color='red')
    
    # Setting the title to the current chain
    plt.title(chain)
    
    # Labeling the axes
    plt.xlabel('Date')
    plt.ylabel('L1 Gas Used Inbox')
    
    # Displaying the legend
    plt.legend()

# Adjusting the layout to prevent overlapping
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
file_path = 'outputs/op_chain_gas_economics_sample.csv'
data = pd.read_csv(file_path)

# Create the 'full_days_data' dataframe by filtering rows where 'active_secs_per_day' is equal to the maximum value
max_active_secs = data['active_secs_per_day'].max()
full_days_data = data[data['active_secs_per_day'] == max_active_secs]

# Create new features representing gas used and calldata bytes per user transaction
full_days_data['l1_gas_used_per_user_tx'] = full_days_data['l1_gas_used_user_txs_l2_per_day'] / full_days_data['l2_num_txs_per_day']
full_days_data['calldata_bytes_per_user_tx'] = full_days_data['calldata_bytes_user_txs_l2_per_day'] / full_days_data['l2_num_txs_per_day']

# Fill NaN values with 0
full_days_data['l1_gas_used_per_user_tx'].fillna(0, inplace=True)
full_days_data['calldata_bytes_per_user_tx'].fillna(0, inplace=True)

# Find the minimum 'l1_gas_used_inbox' value for days with zero transactions and set it as a new feature
min_gas_used_inbox = full_days_data.loc[full_days_data['l2_num_txs_per_day'] == 0, 'l1_gas_used_inbox'].min()
full_days_data['min_gas_used_inbox'] = min_gas_used_inbox

# Define features and target variable
features = [
    'l1_gas_used_per_user_tx',
    'calldata_bytes_per_user_tx',
    'l2_num_txs_per_day', 
    'l2_num_user_deposit_txs_per_day',
    'min_gas_used_inbox'
]
target = 'l1_gas_used_inbox'

# Split the data into training and testing sets
X = full_days_data[features]
y = full_days_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = linear_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the model performance metrics
print(f"MSE: {mse}")
print(f"R²: {r2}")



In [ ]:
# Extracting the coefficients and intercept from the trained model
intercept = linear_regressor.intercept_
coefficients = linear_regressor.coef_

# Creating a dictionary to hold the feature names and their respective coefficients
feature_coeff_dict = dict(zip(features, coefficients))

# Displaying the intercept and coefficients
intercept, feature_coeff_dict

In [ ]:
# Setting up the plot
plt.figure(figsize=(12, 8))

# Adding a column to the testing set to store the predictions
X_test['predicted_l1_gas_used_inbox'] = y_pred

# Merging with the full dataset to get the 'chain' and 'dt' columns for plotting
test_data_with_predictions = full_days_data.merge(X_test[['predicted_l1_gas_used_inbox']], left_index=True, right_index=True)

# List of unique chains
chains = test_data_with_predictions['chain'].unique()

# Looping over each chain to plot the actual vs. predicted values and residuals
for i, chain in enumerate(chains):
    
    # Filtering the data for the current chain
    chain_data = test_data_with_predictions[test_data_with_predictions['chain'] == chain]
    
    # Plotting the actual and predicted values
    plt.subplot(2, 2, i+1)
    plt.plot(chain_data['dt'], chain_data['l1_gas_used_inbox'], label='Actual', marker='o', linestyle='-')
    plt.plot(chain_data['dt'], chain_data['predicted_l1_gas_used_inbox'], label='Predicted', marker='o', linestyle='-')
    
    # Plotting the residuals
    for j in range(len(chain_data)):
        plt.plot([chain_data['dt'].iloc[j], chain_data['dt'].iloc[j]], 
                 [chain_data['l1_gas_used_inbox'].iloc[j], chain_data['predicted_l1_gas_used_inbox'].iloc[j]], 
                 color='gray', linestyle='--', linewidth=0.5)
    
    # Adding labels and title
    plt.xlabel('Date')
    plt.ylabel('L1 Gas Used Inbox')
    plt.title(f'Actual vs Predicted (Chain: {chain})')
    plt.legend()
    plt.grid(True)

# Adjusting the layout to prevent overlapping
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = 'outputs/op_chain_gas_economics_sample.csv'
data = pd.read_csv(file_path)

# Create interaction terms
features = [
    'l2_num_txs_per_day', 
    'l2_num_attr_deposit_txs_per_day', 
    'l2_num_user_deposit_txs_per_day',
    'l1_gas_used_user_txs_l2_per_day',
    'calldata_bytes_user_txs_l2_per_day',
    'l2_gas_used'
]

interaction_terms = []
for i, feature1 in enumerate(features):
    for feature2 in features[i+1:]:
        interaction_term = f"{feature1}_x_{feature2}"
        data[interaction_term] = data[feature1] * data[feature2]
        interaction_terms.append(interaction_term)

# Display the interaction terms
interaction_terms
